In [7]:
with open("inputs/Day_23.txt") as f:
    raw_input_data = f.read()

In [71]:
from collections import namedtuple, defaultdict
from itertools import combinations
from enum import Enum

    
def part_1_solution(raw_input):
    program = list(map(int, raw_input.split(',')))
    
    network = Network(program, 50)
    network.simulate()
    

class Network:
    def __init__(self, program, nbr_of_computers=50):
        self.packets = defaultdict(list)
        self.computers = None
        self.commands = None
        
        self._boot_computers(nbr_of_computers, program)
        
    def simulate(self):
        partial_commands = defaultdict(list)
        
        while True:
            for computer_idx, computer in enumerate(self.computers):
                action = self.commands[computer_idx]
                
                if action is Input:
                    queue = self.packets[computer_idx]
                    if queue:
                        packet = queue[0]
                        to_send = packet.pop(0)
                        if not packet:
                            queue.pop(0)
                    else:
                        to_send = -1
                        
                    next_action = computer.send(to_send)
                elif isinstance(action, Output):
                    output = partial_commands[computer_idx]
                    output.append(action.code)
                    
                    if len(output) >= 3:
                        # packet transmition complete
                        partial_commands[computer_idx] = list()
                        target_idx, x, y = output
                        
                        if target_idx == 255:
                            print(f"Part 1 solution: {y}")
                            return
                        
                        packet = [x, y]
                        self.packets[target_idx].append(packet)
                        
                    next_action = next(computer)
                        
                else:
                    raise Exception(f"Unsuported item from generator: {action}")
                    
                self.commands[computer_idx] = next_action
                
        
    def _boot_computers(self, nbr_of_computers, program):
        self.computers = list()
        self.commands = dict()
    
        for computer_idx in range(nbr_of_computers):
            computer = start_computer(program)
            next(computer)
            action = computer.send(computer_idx)
            self.commands[computer_idx] = action
            self.computers.append(computer)
    
    
Finished = object()
Input = object()
Output = namedtuple('Output', ('code'))


def start_computer(sequence):
    index = 0
    relative_base = 0
    memory = defaultdict(int)
    
    for i, value in enumerate(sequence):
        memory[i] = value
        
    diag_nbr = None
    while True:
        opt_code = memory[index]
        opt_code_with_modes = str(opt_code).zfill(5)
        opt_code = int(opt_code_with_modes[-2:])
        modes = opt_code_with_modes[:-2]
        
        
        par_1_address = get_parameter_address(memory, modes[-1], index + 1, relative_base)
        par_2_address = get_parameter_address(memory, modes[-2], index + 2, relative_base)
        par_3_address = get_parameter_address(memory, modes[-3], index + 3, relative_base)
        
        par_1 = memory[par_1_address]
        par_2 = memory[par_2_address]
        
        if opt_code == 99:
            break
        elif opt_code == 3:
            target_address = par_1_address
            memory[target_address] = yield Input
            index += 2
        elif opt_code == 4:
            raw_output = par_1
            yield Output(raw_output)
            index += 2
        elif opt_code == 1:
            target_address = par_3_address
            memory[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = par_3_address
            memory[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        elif opt_code == 9:
            relative_base += par_1
            index += 2
        else:
            print(f"Wrong code: {opt_code}")  
    
    yield Finished

            
def get_parameter_address(memory, mode, par_index, relative_base):
    if mode == "0":
        return memory[par_index]
    elif mode == "1":
        return par_index
    elif mode == "2":
        return relative_base + memory[par_index]
    else:
        print(f"[ERROR] Wrong mode code: {mode}")

In [72]:
part_1_solution(raw_input_data)

Part 1 solution: 17541


In [69]:
from collections import namedtuple, defaultdict
from itertools import combinations
from enum import Enum

    
def part_2_solution(raw_input):
    program = list(map(int, raw_input.split(',')))
    
    network = Network(program, 50)
    network.simulate()
    

class State(Enum):
    Booting = 0
    Sending = 1
    Receiving = 2
    
    
class Network:
    def __init__(self, program, nbr_of_computers=50):
        self.packets = defaultdict(list)
        self.computers = None
        self.states = defaultdict(lambda: State.Booting)
        self.commands = None
        
        self._boot_computers(nbr_of_computers, program)
        
    def simulate(self):
        partial_commands = defaultdict(list)
        ticks_cnt = 0
        last_nat_y = None
        
        while True:
            for computer_idx, computer in enumerate(self.computers):
                action = self.commands[computer_idx]
                
                if action is Input:
                    queue = self.packets[computer_idx]
                    if queue:
                        packet = queue[0]
                        to_send = packet.pop(0)
                        if not packet:
                            queue.pop(0)
                    else:
                        to_send = -1
                        
                    self.states[computer_idx] = State.Receiving
                    next_action = computer.send(to_send)
                elif isinstance(action, Output):                    
                    output = partial_commands[computer_idx]
                    output.append(action.code)
                    
                    if len(output) >= 3:
                        # packet transmition complete
                        partial_commands[computer_idx] = list()
                        target_idx, x, y = output
                        packet = [x, y]
                        
                        if target_idx == 255:
                            self.packets[target_idx] = packet
                        else:                      
                            self.packets[target_idx].append(packet)
                        
                    self.states[computer_idx] = State.Sending
                    next_action = next(computer)
                        
                else:
                    raise Exception(f"Unsuported item from generator: {action}")
                    
                self.commands[computer_idx] = next_action
                
            ticks_cnt += 1
#             print(ticks_cnt)
#             print(partial_commands)
            if self.is_network_idle():
                nat_packet = self.packets[255][:]
                if not nat_packet:
                    continue
#                 print(nat_packet)
                if last_nat_y == nat_packet[1]:
                    print(f"Part 2 solution: {last_nat_y - 1}")
                    return
                
                last_nat_y = nat_packet[1]
                self.packets[0].append(nat_packet)
                self.packets[255] = list()
                
    def is_network_idle(self):
#         print(self.packets)
#         print(self.states)
        for computer_idx, _ in enumerate(self.computers):
            if self.packets[computer_idx]:
                return False
            
            if self.states[computer_idx] != State.Receiving:
                return False
            
        return True
    
    def _boot_computers(self, nbr_of_computers, program):
        self.computers = list()
        self.commands = dict()
    
        for computer_idx in range(nbr_of_computers):
            computer = start_computer(program)
            next(computer)
            action = computer.send(computer_idx)
            self.commands[computer_idx] = action
            self.computers.append(computer)
    
    
Finished = object()
Input = object()
Output = namedtuple('Output', ('code'))


def start_computer(sequence):
    index = 0
    relative_base = 0
    memory = defaultdict(int)
    
    for i, value in enumerate(sequence):
        memory[i] = value
        
    diag_nbr = None
    while True:
        opt_code = memory[index]
        opt_code_with_modes = str(opt_code).zfill(5)
        opt_code = int(opt_code_with_modes[-2:])
        modes = opt_code_with_modes[:-2]
        
        
        par_1_address = get_parameter_address(memory, modes[-1], index + 1, relative_base)
        par_2_address = get_parameter_address(memory, modes[-2], index + 2, relative_base)
        par_3_address = get_parameter_address(memory, modes[-3], index + 3, relative_base)
        
        par_1 = memory[par_1_address]
        par_2 = memory[par_2_address]
        
        if opt_code == 99:
            break
        elif opt_code == 3:
            target_address = par_1_address
            memory[target_address] = yield Input
            index += 2
        elif opt_code == 4:
            raw_output = par_1
            yield Output(raw_output)
            index += 2
        elif opt_code == 1:
            target_address = par_3_address
            memory[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = par_3_address
            memory[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        elif opt_code == 9:
            relative_base += par_1
            index += 2
        else:
            print(f"Wrong code: {opt_code}")  
    
    yield Finished

            
def get_parameter_address(memory, mode, par_index, relative_base):
    if mode == "0":
        return memory[par_index]
    elif mode == "1":
        return par_index
    elif mode == "2":
        return relative_base + memory[par_index]
    else:
        print(f"[ERROR] Wrong mode code: {mode}")

In [70]:
part_2_solution(raw_input_data)

Part 2 solution: 12415
